In [ ]:
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_10.csv')
df

# Law Article 13

## MEU_13_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：结合所处行业特点、发展阶段和自身经营模式、盈利水平、偿债能力、资金需求等因素，未进行现金分红或现金分红水平较低原因的说明 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2057 |
| completion_tokens | 11085 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_1(df):
    '''
    检查MEU_13_1合规性：
    subject: 上市公司
    condition: 1.合并/母公司资产负债表未分配利润均为正值 2.报告期盈利 3.未分红或三年分红低于三年均利30%
    constraint: 需在权益分派方案中披露分红原因说明
    '''
    
    # 初始化合规列
    df['meu_13_1_subject'] = None  # 所有记录主体均为上市公司
    df['meu_13_1_condition'] = None
    df['meu_13_1_constraint'] = None

    # 条件验证（分三步处理复杂逻辑）
    # 1. 资产负债表校验
    balance_condition = (
        (df['合并报表未分配利润'] > 0) &
        (df['母公司报表未分配利润'] > 0)
    )

    # 2. 盈利能力校验
    profitability_condition = (df['净利润'] > 0)

    # 3. 分红比例校验（需跨期计算）
    def check_dividend_ratio(row):
        try:
            # 获取当前财务年度
            current_year = row['财务数据日期'].year
            
            # 筛选公司近三年年报
            mask = (
                (df['公司简称'] == row['公司简称']) &
                (df['财务数据日期'].dt.year.between(current_year-2, current_year)) &
                (df['财务数据日期'].dt.month == 12)
            )
            annual_data = df[mask].sort_values('财务数据日期')
            
            # 需要完整三年数据
            if len(annual_data) < 3:
                return False
                
            # 计算分红总额和净利润均值
            total_dividend = annual_data['现金分红金额'].sum()
            avg_profit = annual_data['净利润'].mean()
            
            # 判断分红比例条件
            return (total_dividend == 0) or (avg_profit != 0 and total_dividend < avg_profit * 0.3)
        except:
            return False

    df['dividend_condition'] = df.apply(check_dividend_ratio, axis=1)
    
    # 综合条件判断
    df['meu_13_1_condition'] = (
        balance_condition &
        profitability_condition &
        df['dividend_condition']
    )

    # 约束条件验证（使用LLM内容检查）
    constraint_desc = '''结合所处行业特点、发展阶段和自身经营模式、盈利水平、
    偿债能力、资金需求等因素，未进行现金分红或现金分红水平较低原因的说明'''
    
    df.loc[df['meu_13_1_condition'], 'meu_13_1_constraint'] = df[
        df['meu_13_1_condition']
    ]['权益分派方案文本'].apply(
        lambda x: llm_content_check(x, constraint_desc) if pd.notna(x) else False
    )


    return df

In [ ]:
df = check_meu_13_1(df)
df

---

## MEU_13_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：留存未分配利润的预计用途以及收益情况 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2034 |
| completion_tokens | 7261 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_2(df):
    '''
    检验MEU_13_2合规性：
    subject: 上市公司
    condition: 合并及母公司资产负债表未分配利润均为正且报告期盈利，且未分红或近三年分红低于年均净利30%
    constraint: 需在权益分派方案中披露留存利润用途及收益
    '''
    
    # 初始化标记列
    df['meu_13_2_subject'] = True  # 所有行主体均为上市公司
    df['meu_13_2_condition'] = None
    df['meu_13_2_constraint'] = None
    
    # 条件判断预处理
    is_annual_report = df['财务数据日期'].dt.month == 12  # 年报标识
    
    # 条件1: 合并及母公司未分配利润均为正
    profit_condition = (df['合并报表未分配利润'] > 0) & (df['母公司报表未分配利润'] > 0)
    
    # 条件2: 报告期盈利
    profit_report = df['净利润'] > 0
    
    # 条件3: 未分红或近三年分红低于年均净利30% (仅年报处理)
    df['cash_dividend_3y_sum'] = pd.NA
    df['avg_net_profit_3y'] = pd.NA
    
    # 年报数据处理
    annual_mask = is_annual_report & profit_condition & profit_report
    annual_df = df[annual_mask].copy()
    
    if not annual_df.empty:
        # 按公司分组计算三年滚动指标
        annual_df.sort_values(['公司简称', '财务数据日期'], inplace=True)
        
        # 计算三年现金分红总和
        annual_df['cash_dividend_3y_sum'] = annual_df.groupby('公司简称')['现金分红金额']\
            .transform(lambda s: s.rolling(3, min_periods=3).sum())
        
        # 计算三年平均净利润
        annual_df['net_profit_3y_sum'] = annual_df.groupby('公司简称')['净利润']\
            .transform(lambda s: s.rolling(3, min_periods=3).sum())
        annual_df['avg_net_profit_3y'] = annual_df['net_profit_3y_sum'] / 3
        
        # 合并计算结果
        df.update(annual_df[['cash_dividend_3y_sum', 'avg_net_profit_3y']])
        
        # 条件3判断
        cond3 = (
            (~annual_df['是否实施现金分红']) | 
            (annual_df['cash_dividend_3y_sum'] < annual_df['avg_net_profit_3y'] * 0.3)
        )
        df.loc[annual_df.index, 'meu_13_2_condition'] = cond3
    
    # 约束条件检查
    constraint_mask = df['meu_13_2_condition'] & df['meu_13_2_subject']
    df.loc[constraint_mask, 'meu_13_2_constraint'] = df.loc[constraint_mask, '权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "留存未分配利润的预计用途以及收益情况") if pd.notna(x) else False
    )
    
    
    return df

In [ ]:
df = check_meu_13_2(df)
df

---

## MEU_13_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：公司在相应期间是否按照中国证券监督管理委员会相关规定为中小股东参与现金分红决策提供了便利 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2043 |
| completion_tokens | 8462 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_3(df):
    '''
    验证上市公司在满足特定利润条件和分红限制时是否披露中小股东参与决策便利
    "subject": "上市公司",
    "condition": "合并及母公司未分配利润为正+报告期盈利+未分红或三年分红不足净利润30%",
    "constraint": "需在权益分派方案中披露中小股东参与决策便利条款"
    '''
    
    # 初始化合规标记列
    df['meu_13_3_subject'] = True  # 所有行默认符合主体要求
    df['meu_13_3_condition'] = None
    df['meu_13_3_constraint'] = None

    # 预处理财务日期并筛选年报数据
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    annual_mask = df['财务数据日期'].dt.month == 12
    annual_df = df[annual_mask].copy()

    if not annual_df.empty:
        # 按公司分组计算三年滚动指标
        annual_df = annual_df.sort_values(by=['公司简称', '财务数据日期'])
        
        # 计算三年现金分红总和
        annual_df['cash_div_3yr'] = annual_df.groupby('公司简称')['现金分红金额']\
            .transform(lambda s: s.rolling(3, min_periods=3).sum())
        
        # 计算三年平均净利润
        annual_df['net_profit_3yr_avg'] = annual_df.groupby('公司简称')['净利润']\
            .transform(lambda s: s.rolling(3, min_periods=3).sum() / 3)
        
        # 将计算结果合并回原数据框
        df = df.merge(
            annual_df[['cash_div_3yr', 'net_profit_3yr_avg']],
            how='left', left_index=True, right_index=True
        )

        # 条件验证逻辑
        profit_condition = (
            (df['合并报表未分配利润'] > 0) &
            (df['母公司报表未分配利润'] > 0) &
            (df['净利润'] > 0)
        )
        
        dividend_condition = (
            (~df['是否实施现金分红']) |  # 未分红情形
            (df['cash_div_3yr'] < df['net_profit_3yr_avg'] * 0.3)  # 分红不足情形
        )
        
        # 综合条件标记（仅限年报数据）
        df['meu_13_3_condition'] = annual_mask & profit_condition & dividend_condition
        
        # 约束条件验证（仅当满足主体和条件时触发）
        need_check_mask = df['meu_13_3_subject'] & df['meu_13_3_condition']
        df.loc[need_check_mask, 'meu_13_3_constraint'] = df.loc[need_check_mask, '权益分派方案文本'].apply(
            lambda text: llm_content_check(
                text, 
                "公司在相应期间是否按照中国证券监督管理委员会相关规定为中小股东参与现金分红决策提供了便利"
            )
        )

    return df

In [ ]:
df = check_meu_13_3(df)
df

---

## MEU_13_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，未进行现金分红或最近三年现金分红总额低于最近三年年均净利润30% |
| constraint | 应当在权益分派方案中披露：公司为增强投资者回报水平拟采取的措施 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2033 |
| completion_tokens | 9013 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_4(df):
    '''
    验证上市公司在特定财务条件下的信息披露合规性
    "subject": "上市公司",
    "condition": "合并/母公司资产负债表未分配利润正值且盈利，且未分红或三年分红不足",
    "constraint": "需在权益分派方案中披露投资者回报措施"
    '''
    # 初始化标记列
    df['meu_13_4_subject'] = True  # 所有记录均为上市公司
    df['meu_13_4_condition'] = None
    df['meu_13_4_constraint'] = None

    # 条件A: 年末双报表未分配利润正值且当期盈利
    is_year_end = df['财务数据日期'].dt.month == 12
    positive_retained = (df['合并报表未分配利润'] > 0) & (df['母公司报表未分配利润'] > 0)
    profitable = df['净利润'] > 0
    valid_condition_A = is_year_end & positive_retained & profitable

    # 条件B: 未分红或三年分红不足
    # 计算三年分红及利润指标
    annual_mask = df['财务数据日期'].dt.month == 12
    annual_df = df[annual_mask].copy().sort_values(['公司简称', '财务数据日期'])
    
    # 计算滚动三年指标
    annual_df['三年现金分红总额'] = annual_df.groupby('公司简称')['现金分红金额'].transform(
        lambda x: x.rolling(3, min_periods=3).sum())
    annual_df['三年净利润总和'] = annual_df.groupby('公司简称')['净利润'].transform(
        lambda x: x.rolling(3, min_periods=3).sum())
    annual_df['三年年均净利润'] = annual_df['三年净利润总和'] / 3

    # 合并回原始数据
    df = df.merge(
        annual_df[['公司简称', '财务数据日期', '三年现金分红总额', '三年年均净利润']],
        how='left',
        on=['公司简称', '财务数据日期']
    )

    # 构建条件B判断
    no_dividend = ~df['是否实施现金分红']
    insufficient_dividend = (
        df['是否实施现金分红'] & 
        (df['三年现金分红总额'] < df['三年年均净利润'] * 0.3)
    ).fillna(False)
    valid_condition_B = no_dividend | insufficient_dividend

    # 综合条件判断
    df['meu_13_4_condition'] = valid_condition_A & valid_condition_B

    # 约束条件检查
    constraint_desc = "为增强投资者回报水平拟采取的措施"
    df['meu_13_4_constraint'] = df.apply(
        lambda x: llm_content_check(x['权益分派方案文本'], constraint_desc) 
        if x['meu_13_4_condition'] else False,
        axis=1
    )

    return df

In [ ]:
df = check_meu_13_4(df)
df

---

## MEU_13_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 母公司资产负债表中未分配利润为负值但合并资产负债表中未分配利润为正值 |
| constraint | 应当在权益分派相关公告中披露上市公司控股子公司向母公司实施利润分配的情况 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2016 |
| completion_tokens | 5512 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_5(df):
    '''
    检查MEU_13_5合规性：
    - subject: 上市公司（所有记录默认符合）
    - condition: 母公司未分配利润<0且合并报表未分配利润>0
    - constraint: 权益分派公告需披露子公司向母公司利润分配情况
    '''
    
    # 初始化合规标记列
    df['meu_13_5_subject'] = True  # 所有记录主体均为上市公司
    df['meu_13_5_condition'] = None
    df['meu_13_5_constraint'] = None  # 默认视为满足约束
    
    # 标记条件满足情况
    # 条件判断：母公司未分配利润为负且合并报表为正
    valid_condition = (df['母公司报表未分配利润'] < 0) & (df['合并报表未分配利润'] > 0)
    df['meu_13_5_condition'] = valid_condition
    
    # 约束条件处理
    target_description = "上市公司控股子公司向母公司实施利润分配的情况"
    # 仅当同时满足以下情况需要检查：1.条件成立 2.实际进行权益分派
    need_check_mask = valid_condition & df['是否进行权益分派']
    
    # 使用LLM检查分派方案文本
    df.loc[need_check_mask, 'meu_13_5_constraint'] = df.loc[need_check_mask, '权益分派方案文本'].apply(
        lambda text: llm_content_check(text, target_description) if pd.notnull(text) else False
    )
    
    return df

In [ ]:
df = check_meu_13_5(df)
df

---

## MEU_13_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 母公司资产负债表中未分配利润为负值但合并资产负债表中未分配利润为正值 |
| constraint | 应当在权益分派相关公告中披露公司为增强投资者回报水平拟采取的措施 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2016 |
| completion_tokens | 3050 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_13_6(df):
    '''
    验证上市公司在母公司未分配利润为负但合并报表为正时，是否披露增强回报措施
    
    "subject": "上市公司",
    "condition": "母公司资产负债表中未分配利润为负值但合并资产负债表中未分配利润为正值",
    "constraint": "应当在权益分派相关公告中披露公司为增强投资者回报水平拟采取的措施"
    '''
    
    # 初始化所有标记列
    df['meu_13_6_subject'] = True  # 所有记录主体都是上市公司
    df['meu_13_6_condition'] = None
    df['meu_13_6_constraint'] = None
    
    # 1. 验证condition条件
    parent_profit_negative = df['母公司报表未分配利润'] < 0
    consolidated_profit_positive = df['合并报表未分配利润'] > 0
    condition_met = parent_profit_negative & consolidated_profit_positive
    df.loc[condition_met, 'meu_13_6_condition'] = True
    
    # 2. 验证constraint约束（仅当condition满足时检查）
    if condition_met.any():
        # 获取需要检查的公告文本
        target_desc = "为增强投资者回报水平拟采取的措施"
        
        # 对满足条件的行进行内容检查
        constrained_mask = condition_met & df['权益分派方案文本'].notna()
        df.loc[constrained_mask, 'meu_13_6_constraint'] = df.loc[constrained_mask, '权益分派方案文本'].apply(
            lambda text: llm_content_check(text, target_desc)
        )
        
        # 处理文本为空的情况（视为未披露）
        df.loc[condition_met & df['权益分派方案文本'].isna(), 'meu_13_6_constraint'] = False
    
    return df

In [ ]:
df = check_meu_13_6(df)
df

---

# Law Article 14

## MEU_14_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 合并资产负债表、母公司资产负债表中本年末未分配利润均为正值且报告期内盈利，且最近连续两个会计年度经审计的下列财务报表项目金额合计占总资产的50%以上：交易性金融资产、衍生金融资产（套期保值工具除外）、债权投资、其他债权投资、其他权益工具投资、其他非流动金融资产、其他流动资产（待抵扣增值税、预缴税费、合同取得成本等与经营活动相关的资产除外），且未进行现金分红或者现金分红总额低于当年净利润50% |
| constraint | 应当在权益分派方案中结合前述财务报表列报项目的具体情况说明现金分红方案确定的依据及未来增强投资者回报的规划 |
| contextual_info | 财务报表项目具体范围包含：交易性金融资产、衍生金融资产（套期保值工具除外）、债权投资、其他债权投资、其他权益工具投资、其他非流动金融资产、其他流动资产（待抵扣增值税、预缴税费、合同取得成本等与经营活动相关的资产除外） |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2173 |
| completion_tokens | 10263 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_14_1(df):
    '''
    验证上市公司在特定财务条件下的现金分红方案披露合规性
    "subject": "上市公司", 
    "condition": "合并及母公司未分配利润为正+连续两年金融资产占比超50%+现金分红不足净利润50%", 
    "constraint": "需在权益分派方案中说明依据及回报规划", 
    "contextual_info": "金融资产范围包含交易性金融资产等7项（特定项目除外）"
    '''
    # 初始化标记列
    df['meu_14_1_subject'] = True  # 所有行均为上市公司
    df['meu_14_1_condition'] = None
    df['meu_14_1_constraint'] = None

    # 预处理：按公司分组并计算历史数据
    df_sorted = df.sort_values(['公司简称', '财务数据日期'])
    grouped = df_sorted.groupby('公司简称')

    # 计算前一年度财务指标
    financial_columns = ['总资产', '交易性金融资产', '衍生金融资产', '债权投资', '其他债权投资',
                        '其他权益工具投资', '其他非流动金融资产', '其他流动资产']
    for col in financial_columns:
        df_sorted[f'prev_{col}'] = grouped[col].shift(1)

    # 合并回原始索引顺序
    df = df_sorted.reset_index(drop=True).reindex(df.index)

    # 条件验证逻辑
    # 1. 基础财务条件
    is_annual = df['财务数据日期'].dt.month == 12
    profit_condition = (
        (df['合并报表未分配利润'] > 0) &
        (df['母公司报表未分配利润'] > 0) &
        (df['净利润'] > 0)
    )

    # 2. 金融资产占比条件
    current_fa = (
        df['交易性金融资产'] + 
        df['衍生金融资产'] +  # 假设已排除套期工具
        df['债权投资'] +
        df['其他债权投资'] +
        df['其他权益工具投资'] +
        df['其他非流动金融资产'] +
        df['其他流动资产']  # 假设已排除特定项目
    )
    prev_fa = (
        df['prev_交易性金融资产'] + 
        df['prev_衍生金融资产'] +
        df['prev_债权投资'] +
        df['prev_其他债权投资'] +
        df['prev_其他权益工具投资'] +
        df['prev_其他非流动金融资产'] +
        df['prev_其他流动资产']
    )
    fa_ratio_condition = (
        (current_fa / df['总资产'] >= 0.5) &
        (prev_fa / df['prev_总资产'] >= 0.5) &
        df['prev_总资产'].notna()
    )

    # 3. 现金分红条件
    dividend_condition = (
        ~df['是否实施现金分红'] | 
        (df['现金分红金额'] < df['净利润'] * 0.5)
    )

    # 综合条件判断
    valid_condition = (
        is_annual &
        profit_condition &
        fa_ratio_condition &
        dividend_condition
    )
    df['meu_14_1_condition'] = valid_condition

    # 约束条件验证
    constraint_desc = "结合前述财务报表列报项目的具体情况说明现金分红方案确定的依据及未来增强投资者回报的规划"
    need_check = valid_condition & df['是否进行权益分派']
    df.loc[need_check, 'meu_14_1_constraint'] = df.loc[need_check, '权益分派方案文本'].apply(
        lambda x: llm_content_check(x, constraint_desc) if pd.notnull(x) else False
    )


    return df

In [ ]:
df = check_meu_14_1(df)
df

---

# Law Article 15

## MEU_15_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 权益分派方案中现金分红的金额达到或者超过当期净利润的100%且达到或者超过当期末未分配利润的50% |
| constraint | 应当同时披露是否影响偿债能力、过去十二个月内是否使用过募集资金补充流动资金以及未来十二个月内是否计划使用募集资金补充流动资金等内容 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2041 |
| completion_tokens | 5042 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_15_1(df):
    '''
    验证上市公司在超高比例分红时的信息披露合规性
    subject: 上市公司
    condition: 现金分红≥净利润100%且≥未分配利润50%
    constraint: 需披露偿债影响、募集资金使用情况
    '''
    
    # 初始化合规标记列
    df['meu_15_1_subject'] = True  # 所有记录主体均为上市公司
    df['meu_15_1_condition'] = None
    df['meu_15_1_constraint'] = None
    

    # 条件验证：现金分红达标
    cash_condition = (
        (df['现金分红金额'] >= df['净利润']) & 
        (df['现金分红金额'] >= df['合并报表未分配利润'] * 0.5)
    )
    df.loc[cash_condition, 'meu_15_1_condition'] = True
    
    # 约束验证：信息披露完整性
    def check_disclosure(text):
        if pd.isna(text):
            return False
        return (
            llm_content_check(text, "是否影响偿债能力") &
            llm_content_check(text, "过去十二个月内是否使用过募集资金补充流动资金") &
            llm_content_check(text, "未来十二个月内是否计划使用募集资金补充流动资金")
        )
    
    df.loc[cash_condition, 'meu_15_1_constraint'] = df['权益分派方案文本'].apply(check_disclosure)
    
    
    return df

In [ ]:
df = check_meu_15_1(df)
df

---

## MEU_15_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 最近一个会计年度的财务会计报告被出具非无保留意见的审计报告或者带与持续经营相关的重大不确定性段落的无保留意见的审计报告，且实施现金分红的 |
| constraint | 应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2052 |
| completion_tokens | 4557 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_15_2(df):
    '''
    合规性检验函数：MEU_15_2
    "subject": "上市公司",
    "condition": "最近一个会计年度的财务会计报告被出具非无保留意见的审计报告或者带与持续经营相关的重大不确定性段落的无保留意见的审计报告，且实施现金分红的",
    "constraint": "应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营"
    '''

    # 初始化标记列
    df['meu_15_2_subject'] = True  # 所有行均为上市公司
    df['meu_15_2_condition'] = None
    df['meu_15_2_constraint'] = None 

    # 标记条件1：审计意见类型
    audit_condition = df['审计意见'].isin(['保留意见', '非标准无保留意见'])
    # 标记条件2：实施现金分红
    dividend_condition = df['是否实施现金分红']
    # 合并条件
    valid_condition = audit_condition & dividend_condition
    df['meu_15_2_condition'] = valid_condition

    # 标记约束（仅当满足主体和条件时检查）
    constraint_description = "根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营"
    mask = df['meu_15_2_subject'] & df['meu_15_2_condition']
    
    # 对满足条件的行进行内容检查
    df.loc[mask, 'meu_15_2_constraint'] = df.loc[mask].apply(
        lambda row: llm_content_check(row['权益分派方案文本'], constraint_description) if pd.notna(row['权益分派方案文本']) else False,
        axis=1
    )

    return df

In [ ]:
df = check_meu_15_2(df)
df

---

## MEU_15_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 报告期末资产负债率超过80%且当期经营活动产生的现金流量净额为负，且现金分红金额超过当期净利润50%的 |
| constraint | 应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2040 |
| completion_tokens | 3297 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_15_3(df):
    '''
    验证MEU_15_3合规性：
    "subject": "上市公司",
    "condition": "报告期末资产负债率超过80%且当期经营活动产生的现金流量净额为负，且现金分红金额超过当期净利润50%的",
    "constraint": "应当根据公司盈利能力、融资能力及其成本、偿债能力及现金流等情况披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营"
    '''
    
    # 初始化标记列
    df['meu_15_3_subject'] = True  # 所有记录主体均为上市公司
    df['meu_15_3_condition'] = None
    df['meu_15_3_constraint'] = None


    # 1. 验证condition条件
    cond_leverage = df['资产负债率'] > 0.8  # 资产负债率超过80%
    cond_cashflow = df['经营活动现金流量净额'] < 0  # 经营活动现金流为负
    cond_dividend = df['现金分红金额'] > (df['净利润'] * 0.5)  # 分红超净利润50%
    
    valid_condition = cond_leverage & cond_cashflow & cond_dividend
    df.loc[valid_condition, 'meu_15_3_condition'] = True

    # 2. 验证constraint条件（仅当condition满足时检查）
    constraint_desc = '''披露现金分红方案的合理性，是否导致公司营运资金不足或者影响公司正常生产经营，
                         包括对公司盈利能力、融资能力及其成本、偿债能力及现金流等情况的说明'''
    
    # 仅对触发条件的记录进行内容检查
    mask = valid_condition & df['是否实施现金分红']
    df.loc[mask, 'meu_15_3_constraint'] = df[mask].apply(
        lambda x: llm_content_check(x['权益分派方案文本'], constraint_desc), 
        axis=1
    )

    return df

In [ ]:
df = check_meu_15_3(df)
df

---

# Law Article 18

## MEU_18_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且最近两年同期净利润持续增长 |
| constraint | 每股送转股比例不得高于上市公司最近两年同期净利润的复合增长率 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_31_2;MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2176 |
| completion_tokens | 10773 |


### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_meu_18_1(df):
    # 创建副本避免修改原数据
    df = df.copy()
    
    # 初始化合规标记列
    df['meu_18_1_subject'] = True  # 所有行均为上市公司主体
    df['meu_18_1_condition'] = None
    df['meu_18_1_constraint'] = None

    # 条件1: 高送转方案判断
    df['送转合计'] = df['送股比例'] + df['转增比例']
    high_send = (df['送转合计'] >= 5) & df['是否进行权益分派']

    # 条件2: 净利润持续增长判断
    # 生成年季信息
    df['year'] = df['财务数据日期'].dt.year
    df['quarter'] = df['财务数据日期'].dt.quarter
    
    # 获取前两年同期数据
    prev_df = df[['公司简称', 'year', 'quarter', '净利润']].copy()
    prev_df['year'] += 1
    prev_df.rename(columns={'净利润': 'prev_net'}, inplace=True)
    df = df.merge(prev_df, on=['公司简称', 'year', 'quarter'], how='left')

    prev_prev_df = df[['公司简称', 'year', 'quarter', '净利润']].copy()
    prev_prev_df['year'] += 2
    prev_prev_df.rename(columns={'净利润': 'prev_prev_net'}, inplace=True)
    df = df.merge(prev_prev_df, on=['公司简称', 'year', 'quarter'], how='left')

    # 计算连续增长条件
    growth_condition = (df['净利润'] > df['prev_net']) & (df['prev_net'] > df['prev_prev_net'])
    df['meu_18_1_condition'] = high_send & growth_condition.fillna(False)

    # 约束条件: 送转比例<=复合增长率
    # 计算复合增长率
    denominator = df['prev_prev_net'].abs().replace(0, np.nan)
    ratio = df['净利润'] / denominator
    compound_growth = np.sqrt(ratio.where(ratio > 0)) - 1
    
    # 计算送转比例
    send_ratio = df['送转合计'] / 10
    
    # 标记约束有效性
    valid_constraint = send_ratio <= compound_growth
    df['meu_18_1_constraint'] = df['meu_18_1_condition'] & valid_constraint.fillna(False)

    # 清理中间列
    df.drop(columns=['送转合计', 'year', 'quarter', 'prev_net', 'prev_prev_net'], inplace=True)
    
    return df

In [ ]:
df = check_meu_18_1(df)
df

---

## MEU_18_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且在报告期内实施再融资、并购重组等导致净资产有较大变化 |
| constraint | 每股送转股比例不得高于上市公司报告期末净资产较之于期初净资产的增长率 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2110 |
| completion_tokens | 7418 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_2(df):
    '''
    合规性检查函数：MEU_18_2
    subject: 上市公司
    condition: 披露高送转方案且在报告期内实施再融资、并购重组等导致净资产有较大变化
    constraint: 每股送转股比例不得高于上市公司报告期末净资产较之于期初净资产的增长率
    '''
    # 初始化标记列
    df['meu_18_2_subject'] = True  # 所有行均为上市公司
    df['meu_18_2_condition'] = None
    df['meu_18_2_constraint'] = None

    # 条件验证：高送转方案 (每10股送转合计≥5股)
    high_send_condition = (df['送股比例'] + df['转增比例']) >= 5

    # 条件验证：再融资/并购重组事件（通过公告类型判断）
    event_pattern = r'增发|配股|并购|重组|非公开发行|重大资产'
    has_refinance = df['公告类型'].str.contains(event_pattern, case=False, na=False)

    # 合并条件验证结果
    df['meu_18_2_condition'] = high_send_condition & has_refinance

    # 约束验证：计算实际净资产增长率
    df_sorted = df.sort_values(['公司简称', '财务数据日期'])
    df_sorted['prev_净资产'] = df_sorted.groupby('公司简称')['净资产'].shift(1)
    df_sorted['实际净资产增长率'] = (df_sorted['净资产'] - df_sorted['prev_净资产']) / df_sorted['prev_净资产']
    df = df_sorted.reset_index(drop=True)

    # 计算送转股比例约束
    total_send_ratio = (df['送股比例'] + df['转增比例']) / 10
    df['meu_18_2_constraint'] = total_send_ratio <= df['实际净资产增长率']

    # 清理临时列
    df.drop(['prev_净资产', '实际净资产增长率'], axis=1, errors='ignore')
    
    return df

In [ ]:
df = check_meu_18_2(df)
df

---

## MEU_18_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 披露高送转方案且最近两年净利润持续增长且最近三年每股收益均不低于1元且上市公司认为确有必要且依据年度财务报表进行高送转 |
| constraint | 应当充分披露高送转的主要考虑及其合理性且送转股后每股收益不低于0.5元 |
| contextual_info | nan |
| note | "每股收益"简化为采用"每股净利润" |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2136 |
| completion_tokens | 7837 |


### 代码实现

In [ ]:
import pandas as pd
from utils.tool_for_agents import llm_content_check

def check_meu_18_3(df):
    '''
    验证上市公司高送转方案的合规性：
    "subject": "上市公司",
    "condition": "披露高送转方案且最近两年净利润持续增长且最近三年每股收益均不低于1元且上市公司认为确有必要且依据年度财务报表进行高送转",
    "constraint": "应当充分披露高送转的主要考虑及其合理性且送转股后每股收益不低于0.5元"
    '''
    
    # 初始化合规标记列
    df['meu_18_3_subject'] = True  # 所有记录主体均为上市公司
    df['meu_18_3_condition'] = None
    df['meu_18_3_constraint'] = None

    # 转换财务数据日期格式
    df['财务数据日期'] = pd.to_datetime(df['财务数据日期'])
    
    # ======================
    # 条件验证部分
    # ======================
    
    # 1. 年度报表标识（假设12月为年报月份）
    is_annual_report = df['财务数据日期'].dt.month == 12
    
    # 2. 高送转方案验证（每10股送转≥5股）
    sz_ratio = df['送股比例'] + df['转增比例']
    high_sz = sz_ratio >= 5
    
    # 3. 净利润持续增长验证
    df['prev_year_net'] = df.groupby('公司简称')['净利润'].shift(1)
    df['prev_prev_year_net'] = df.groupby('公司简称')['净利润'].shift(2)
    net_growth = (df['净利润'] > df['prev_year_net']) & (df['prev_year_net'] > df['prev_prev_year_net'])
    
    # 4. 每股收益验证（三年EPS≥1）
    df['eps_prev1'] = df.groupby('公司简称')['每股净利润'].shift(1)
    df['eps_prev2'] = df.groupby('公司简称')['每股净利润'].shift(2)
    eps_valid = (df['每股净利润'] >= 1) & (df['eps_prev1'] >= 1) & (df['eps_prev2'] >= 1)
    
    # 5. 必要性验证（LLM内容检查）
    necessity_check = df['权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "上市公司认为确有必要实施高送转") if pd.notnull(x) else False
    )
    
    # 6. 权益分派执行标记
    is_distributed = df['是否进行权益分派']
    
    # 综合条件判断
    condition_met = (
        is_annual_report &
        high_sz &
        net_growth &
        eps_valid &
        necessity_check &
        is_distributed
    )
    df.loc[condition_met, 'meu_18_3_condition'] = True
    
    # ======================
    # 约束验证部分
    # ======================
    
    # 1. 合理性披露检查
    disclosure_check = df['权益分派方案文本'].apply(
        lambda x: llm_content_check(x, "高送转的主要考虑及其合理性") if pd.notnull(x) else False
    )
    
    # 2. 送转后EPS计算
    expansion_ratio = sz_ratio / 10  # 转换为股本扩张比例
    post_eps = df['每股净利润'] / (1 + expansion_ratio)
    eps_constraint = post_eps >= 0.5
    
    # 综合约束判断
    constraint_met = disclosure_check & eps_constraint
    df.loc[constraint_met, 'meu_18_3_constraint'] = True
    
    
    # 清理中间列
    df.drop(columns=['prev_year_net', 'prev_prev_year_net', 'eps_prev1', 'eps_prev2'], 
            inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_18_3(df)
df

---

# Law Article 19

## MEU_19_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 报告期尚未产生收入 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2083 |
| completion_tokens | 4630 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_1(df):
    '''
    检查MEU_19_1合规性：
    - subject: 上市公司（所有记录默认满足）
    - condition: 报告期尚未产生收入（使用净利润<=0近似判断）
    - constraint: 未披露高送转方案（送转合计<5股）
    '''

    # 初始化标记列
    df['meu_19_1_subject'] = True  # 所有记录均为上市公司
    df['meu_19_1_condition'] = None
    df['meu_19_1_constraint'] = None

    # 标记condition：报告期尚未产生收入（使用净利润<=0近似判断）
    # 注意：法律条文中的"产生收入"应指营业收入，但数据中无该字段，此处用净利润作为替代指标
    df['meu_19_1_condition'] = df['净利润'] <= 0

    # 标记constraint：未披露高送转方案
    # 计算送转合计（处理空值为0）
    total_distribution = df['送股比例'].fillna(0) + df['转增比例'].fillna(0)
    # 判断是否达到高送转标准（每10股送转≥5股）
    is_high_distribution = total_distribution >= 5
    # 约束成立条件：未进行权益分派 或 分派但未达高送转标准
    df['meu_19_1_constraint'] = ~(df['是否进行权益分派'] & is_high_distribution)

    return df

In [ ]:
df = check_meu_19_1(df)
df

---

## MEU_19_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 净利润为负 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2081 |
| completion_tokens | 4348 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_2(df):
    '''
    检查MEU_19_2合规性：
    subject: 上市公司
    condition: 净利润为负
    constraint: 不得披露高送转方案
    contextual_info: 高送转定义参考MEU_17_1（每10股合计送转≥5股）
    '''
    # 初始化标记列
    df['meu_19_2_subject'] = True  # 数据主体均为上市公司
    df['meu_19_2_condition'] = None
    df['meu_19_2_constraint'] = None

    # 标记条件满足情况 --------------------------------------------------------
    # 1.标记subject（所有行已默认True）
    
    # 2.标记condition（净利润为负）
    df['meu_19_2_condition'] = df['净利润'] < 0
    
    # 3.标记constraint（未披露高送转）
    # 计算高送转条件：进行权益分派且送转合计≥5股/10股
    high_send_condition = (
        df['是否进行权益分派'] & 
        ((df['送股比例'] + df['转增比例']) >= 5)
    )
    df['meu_19_2_constraint'] = ~high_send_condition  # 约束满足=未实施高送转

    return df

In [ ]:
df = check_meu_19_2(df)
df

---

## MEU_19_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 净利润同比下降50%以上 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2083 |
| completion_tokens | 3810 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_3(df):
    '''
    验证MEU_19_3合规性:
    - subject: 上市公司（所有记录默认满足）
    - condition: 净利润同比下降50%以上（净利润增长率<=-0.5）
    - constraint: 不得披露高送转方案（送股+转增<5股）
    '''

    # 初始化标记列
    df['meu_19_3_subject'] = True  # 所有记录主体均为上市公司
    df['meu_19_3_condition'] = None
    df['meu_19_3_constraint'] = None

    # 标记条件满足情况
    # 1. 验证净利润同比下降50%以上（单季度同比）
    df['meu_19_3_condition'] = df['净利润增长率'] <= -0.5

    # 2. 验证高送转约束（仅当主体和条件同时满足时检查）
    # 计算送转总和（处理可能的NaN值）
    send_ratio = df['送股比例'].fillna(0)
    transfer_ratio = df['转增比例'].fillna(0)
    high_send = (send_ratio + transfer_ratio) >= 5

    # 确定需要检查约束的有效范围
    mask = df['meu_19_3_subject'] & df['meu_19_3_condition']
    # 在约束范围内标记违规（存在高送转）
    df.loc[mask, 'meu_19_3_constraint'] = ~high_send[mask]

    return df

In [ ]:
df = check_meu_19_3(df)
df

---

## MEU_19_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 送转股后每股收益低于0.2元 |
| constraint | 不得披露高送转方案 |
| contextual_info | nan |
| note | "每股收益"简化为采用"每股净利润" |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2098 |
| completion_tokens | 7258 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_4(df):
    '''
    检查上市公司在送转股后每股收益低于0.2元时是否违规披露高送转方案。
    MEU内容:
    - subject: 上市公司
    - condition: 送转股后每股收益低于0.2元
    - constraint: 不得披露高送转方案
    '''

    # 初始化标记列
    df['meu_19_4_subject'] = True  # 所有行主体均为上市公司
    df['meu_19_4_condition'] = None
    df['meu_19_4_constraint'] = None

    # 计算高送转条件（每10股送转合计≥5股）
    total_sz_ratio = df['送股比例'] + df['转增比例']
    is_high_sz = total_sz_ratio >= 5

    # 计算送转后每股收益（考虑股本扩张效应）
    df['post_eps'] = df['每股净利润'] * 10 / (10 + total_sz_ratio)

    # 条件验证：实际实施高送转且送转后EPS<0.2
    valid_condition = (
        df['是否进行权益分派'] & 
        is_high_sz & 
        (df['post_eps'] < 0.2)
    )
    df['meu_19_4_condition'] = valid_condition

    # 约束验证：当条件成立时不得实施高送转
    df['meu_19_4_constraint'] = ~valid_condition

    # 清理中间列
    df.drop(columns=['post_eps'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_19_4(df)
df

---

## MEU_19_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 相关股东在前三个月存在减持情形 |
| constraint | 不得披露高送转方案 |
| contextual_info | 相关股东指公司的提议股东和控股股东及其一致行动人、董事、监事及高级管理人员 |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2105 |
| completion_tokens | 2801 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_5(df):
    '''
    验证MEU_19_5合规性:
    - subject: 上市公司
    - condition: 相关股东前三个月存在减持情形
    - constraint: 不得披露高送转方案
    '''

    # 初始化标记列
    df['meu_19_5_subject'] = True  # 所有记录主体均为上市公司
    df['meu_19_5_condition'] = df['相关股东前三个月内存在减持情形']
    df['meu_19_5_constraint'] = None

    # 计算高送转方案指标（每10股合计送转≥5股）
    total_distribution = df['送股比例'] + df['转增比例']
    is_high_send = total_distribution >= 5
    
    # 判断是否实际披露高送转（需同时满足权益分派和送转比例条件）
    has_high_send_plan = df['是否进行权益分派'] & is_high_send
    df['meu_19_5_constraint'] = ~has_high_send_plan


    return df

In [ ]:
df = check_meu_19_5(df)
df

---

## MEU_19_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 相关股东在后三个月存在减持计划 |
| constraint | 不得披露高送转方案 |
| contextual_info | 相关股东指公司的提议股东和控股股东及其一致行动人、董事、监事及高级管理人员 |
| note | nan |
| relation | refer_to |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 2105 |
| completion_tokens | 5459 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_6(df):
    '''
    验证上市公司在相关股东存在未来三个月减持计划时是否违规披露高送转方案
    MEU_19_6参数:
    - subject: 上市公司
    - condition: 相关股东在后三个月存在减持计划
    - constraint: 不得披露高送转方案
    - contextual_info: 相关股东定义及高送转标准参考
    '''

    # 初始化合规标记列
    df['meu_19_6_subject'] = True  # 数据源默认均为上市公司
    df['meu_19_6_condition'] = None
    df['meu_19_6_constraint'] = None


    # 1. 标记valid的subject（所有行默认满足）
    # 根据数据特性，假设所有记录主体均为上市公司

    # 2. 标记valid的condition
    df['meu_19_6_condition'] = df['相关股东后三个月存在减持计划']

    # 3. 标记valid的constraint
    # 计算高送转标准（每10股合计≥5股）
    high_song_zhuan = (df['送股比例'] + df['转增比例']) >= 5
    # 结合权益分派实施状态
    high_song_zhuan = df['是否进行权益分派'] & high_song_zhuan
    # 约束有效性判断：当condition为True时不得存在高送转
    df['meu_19_6_constraint'] = (~high_song_zhuan) | (~df['meu_19_6_condition'])

    return df

In [ ]:
df = check_meu_19_6(df)
df

---